### Actividad: patrones de diseño para microservicios


Los patrones de diseño para microservicios que son especialmente útiles en entornos de computación concurrente, paralela y distribuida abordan principalmente cómo descomponer las aplicaciones en servicios más pequeños, independientes y escalables. Estos patrones también se centran en cómo estos servicios pueden comunicarse de manera eficiente, manejar fallos y procesar datos de manera concurrente y paralela. 

- Service Discovery: En un sistema distribuido con muchos microservicios, es crucial que los servicios puedan encontrarse y comunicarse entre sí. El patrón de Service Discovery permite a los microservicios registrarse en un servicio central y descubrir dinámicamente la ubicación de otros servicios necesarios para completar una solicitud.

- Circuit Breaker: Este patrón previene fallos en un microservicio que puede causar fallas en cascada a otros servicios dependientes. Al detectar fallas repetidas en un servicio, el Circuit Breaker corta temporalmente la llamada al servicio fallido, permitiendo que se recupere y evitando que el sistema completo se vea afectado.

- Bulkhead: Inspirado en los compartimientos estancos de un barco, este patrón limita las fallas a partes aisladas del sistema para evitar que se propaguen. En el contexto de la computación paralela y distribuida, este patrón puede ser implementado para limitar la cantidad de recursos que un solo cliente o servicio puede consumir, permitiendo así que el sistema mantenga la estabilidad bajo carga.

- Sidecar: Este patrón se utiliza para desacoplar aspectos de la infraestructura de los microservicios principales, permitiendo que cada servicio se enfoque en su lógica de negocio. Un proceso sidecar puede manejar funcionalidades como monitoreo, registro, configuración de red, que son esenciales para la computación distribuida y la comunicación entre servicios.

- Event Sourcing: En este patrón, los cambios en el estado de la aplicación son registrados como una secuencia de eventos. Esto permite que el sistema sea más resiliente y escalable, facilitando el procesamiento paralelo de eventos y la reconstrucción del estado del sistema a partir de estos eventos si es necesario.
  
- Sagas: Para manejar transacciones que involucran múltiples microservicios en un entorno distribuido, el patrón de Sagas descompone las transacciones en una serie de operaciones locales en cada servicio. Si una operación falla, se ejecutan compensaciones en los servicios afectados para revertir la transacción.

- Edge Server:  actúa como un intermediario entre los clientes externos y tu red de microservicios. Este patrón es particularmente útil para manejar preocupaciones comunes como seguridad, balanceo de carga, autenticación y enrutamiento. El Edge Server puede simplificar la interacción del cliente con el conjunto de microservicios y puede actuar como un punto de control para optimizar las solicitudes antes de que lleguen a los servicios internos.

- Reactive Microservices: son diseñados siguiendo los principios de la programación reactiva, lo que significa que son construidos para ser no bloqueantes, orientados a eventos y capaces de manejar un alto grado de procesamiento concurrente y flujos de datos en tiempo real. Esto les permite responder de manera más eficiente a las interacciones con los usuarios o con otros servicios, mejorando el rendimiento y la escalabilidad.

- Distributed Tracing:  crucial en arquitecturas de microservicios donde las solicitudes pueden cruzar múltiples servicios antes de completarse. Este patrón ayuda a monitorear y diagnosticar problemas en aplicaciones distribuidas al proporcionar visibilidad en cómo las solicitudes viajan a través de los servicios. 


#### Ejemplo de implementación de Service Discovery

Para este ejemplo, necesitarás tener instalado Flask y requests. Puedes instalarlos mediante pip si no los tienes:

pip install Flask requests


**Servicio de descubrimiento de servicios**

El servicio de descubrimiento mantendrá un registro de las instancias de microservicios y proporcionará un endpoint para que los clientes descubran y se comuniquen con las instancias disponibles.



In [1]:
from flask import Flask, jsonify, request
import requests

app = Flask(__name__)

# Registro de microservicios disponibles
services = {
    'microservice_a': ['http://127.0.0.1:5001', 'http://127.0.0.1:5002', 'http://127.0.0.1:5003']
}

# Endpoint para registrar/desregistrar servicios
@app.route('/register', methods=['POST'])
def register_service():
    data = request.json
    if data['action'] == 'register':
        if data['service_name'] not in services:
            services[data['service_name']] = []
        if data['url'] not in services[data['service_name']]:
            services[data['service_name']].append(data['url'])
    elif data['action'] == 'unregister':
        if data['service_name'] in services and data['url'] in services[data['service_name']]:
            services[data['service_name']].remove(data['url'])
    return jsonify(services)

# Endpoint para obtener una instancia disponible
@app.route('/discover/<service_name>')
def discover_service(service_name):
    if service_name in services and services[service_name]:
        # Simple round-robin load balancing
        url = services[service_name].pop(0)
        services[service_name].append(url)
        return jsonify({'url': url})
    return jsonify({'error': 'No service available'}), 404

if __name__ == '__main__':
    app.run(port=5000, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/andersonrojas/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/andersonrojas/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/andersonrojas/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/home/andersonrojas/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/andersonrojas/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", l

SystemExit: 1

/home/andersonrojas/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**Microservicios**

Aquí tienes un ejemplo simple de microservicios que se registran automáticamente en el servicio de descubrimiento cuando se inician.

In [2]:
from flask import Flask
import requests

app = Flask(__name__)

service_url = 'http://127.0.0.1:5001'  # Cambia el puerto para cada instancia
service_name = 'microservice_a'

@app.route('/')
def home():
    return "Hola desde Microservicio A en " + service_url

if __name__ == '__main__':
    # Registrar en el servicio de descubrimiento
    requests.post('http://127.0.0.1:5000/register', json={'action': 'register', 'service_name': service_name, 'url': service_url})
    app.run(port=int(service_url.split(':')[-1]))


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /register (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ae1abf616c0>: Failed to establish a new connection: [Errno 111] Connection refused'))

En este ejemplo, el service discovery está implementado de forma muy básica con balanceo de carga round-robin. Cada instancia de microservicio se registra automáticamente al iniciarse.

Cuando un cliente quiere interactuar con un microservicio, primero contacta al servicio de descubrimiento para obtener la URL de una instancia activa. Una vez que tiene la URL, el cliente puede enviar directamente sus solicitudes a esa instancia de microservicio.

Este flujo asegura que:

- Los microservicios pueden registrarse y desregistrarse de manera dinámica.
- Los clientes siempre tienen acceso a instancias activas de microservicios.
- Las cargas se distribuyen equitativamente entre todas las instancias disponibles.

Este enfoque también ayuda a manejar la escalabilidad, ya que nuevas instancias pueden ser añadidas o retiradas sin afectar a los clientes, siempre y cuando utilicen el servicio de descubrimiento para obtener las URLs activas. Además, se puede mejorar la robustez del sistema implementando chequeos de salud regular en cada microservicio, permitiendo que el servicio de descubrimiento retire automáticamente cualquier instancia que no responda adecuadamente.

**Ejecución del código** 

Servicio de descubrimiento de Servicios:
- Guarda el código del servicio de descubrimiento en un archivo, por ejemplo, service_discovery.py.
- Ejecuta este archivo en una terminal. Este servicio correrá en el puerto 5000

python service_discovery.py


Microservicios:
Guarda el código del microservicio en otro archivo, por ejemplo, microservice_a.py.

- Modifica la variable service_url para cada instancia que quieras correr. Por ejemplo, puedes usar 5001, 5002, etc., para diferentes instancias.
- Ejecuta cada instancia en su propia terminal. Asegúrate de cambiar el puerto cada vez que ejecutes una nueva instancia:

python microservice_a.py

Repite este paso para cada instancia que desees ejecutar, asegurándote de cambiar service_url para que cada una escuche en un puerto diferente.

#### Testeo de funcionamiento

Verifica el registro de servicios:

- Puedes verificar qué servicios están registrados enviando una solicitud GET a http://127.0.0.1:5000/register. Puedes usar un navegador o herramientas como Postman o curl para esto.

Descubrimiento de servicios:

- Para obtener la URL de una instancia activa de microservice_a, puedes enviar una solicitud GET a http://127.0.0.1:5000/discover/microservice_a. De nuevo, puedes usar un navegador o herramientas de API para verificar esto.

Envía solicitudes a las instancias del microservicio:

- Usando la URL proporcionada por el servicio de descubrimiento, puedes enviar solicitudes directamente a las instancias activas del microservicio. Por ejemplo, si obtienes http://127.0.0.1:5001 como respuesta, puedes acceder a esta dirección en tu navegador para ver la respuesta del microservicio.

#### Tu participación

- ¿Qué es un servicio de descubrimiento en el contexto de microservicios y por qué es crucial para la operación eficiente de una arquitectura basada en microservicios?

- Implementa una tercera instancia de microservice_a que escuche en el puerto 5003. Regístrala en el servicio de descubrimiento y verifica que el balanceo de carga funcione correctamente enviando múltiples solicitudes a /discover/microservice_a.
  * Modifica microservice_a.py para cambiar el puerto a 5003.
  * Ejecuta el servicio y observa cómo se comporta el balanceo de carga.

- Simula una falla en una de las instancias del microservicio (por ejemplo, deteniendo el proceso que corre en el puerto 5001) y observa cómo el servicio de descubrimiento maneja la ausencia de la instancia fallida. ¿Continúa dirigiendo solicitudes a la instancia detenida?
  * Detén la instancia del microservicio en el puerto 5001.
  * Envía solicitudes a /discover/microservice_a y verifica si la instancia detenida sigue siendo ofrecida por el servicio de descubrimiento.

- Escribe un script que automáticamente desregistre una instancia del microservicio del servicio de descubrimiento cuando detecte un error o una interrupción en la instancia.
    * Implementa una verificación de salud en microservice_a.py.
    * Usa una ruta como /health para reportar el estado de salud del servicio.
    * Modifica el servicio de descubrimiento para eliminar automáticamente servicios que no pasen la verificación de salud.



In [ ]:
from flask import Flask, jsonify, request
import requests

app = Flask(__name__)

services = {
    'microservice_a': ['http://127.0.0.1:5001', 'http://127.0.0.1:5002', 'http://127.0.0.1:5003']
}

@app.route('/register', methods=['POST'])
def register_service():
    data = request.json
    if data['action'] == 'register':
        if data['service_name'] not in services:
            services[data['service_name']] = []
        if data['url'] not in services[data['service_name']]:
            services[data['service_name']].append(data['url'])
    elif data['action'] == 'unregister':
        if data['service_name'] in services and data['url'] in services[data['service_name']]:
            services[data['service_name']].remove(data['url'])
    return jsonify(services)

@app.route('/discover/<service_name>')
def discover_service(service_name):
    available_services = []
    if service_name in services:
        for url in services[service_name]:
            if requests.get(url + '/health').status_code == 200:
                available_services.append(url)
        if available_services:
            url = available_services.pop(0)
            available_services.append(url)
            return jsonify({'url': url})
    return jsonify({'error': 'No service available'}), 404

if __name__ == '__main__':
    app.run(port=5000, debug=True)


### Ejemplo de implementación del Circuit Breaker
Primero, crearemos una clase CircuitBreaker para manejar el estado del circuito y la lógica de transición entre estados (cerrado, abierto y semiabierto).

In [ ]:
import time
from flask import Flask, jsonify

app = Flask(__name__)

class CircuitBreaker:
    def __init__(self, fail_threshold=3, reset_timeout=10):
        self.fail_threshold = fail_threshold
        self.reset_timeout = reset_timeout
        self.failures = 0
        self.last_failure_time = None
        self.state = "closed"

    def call(self, func, *args, **kwargs):
        if self.state == "open":
            if time.time() - self.last_failure_time > self.reset_timeout:
                self.state = "half-open"
            else:
                raise Exception("Circuit is open. Calls are not allowed.")
        
        try:
            result = func(*args, **kwargs)
            self.reset()
            return result
        except Exception as e:
            self.record_failure()
            raise e

    def record_failure(self):
        self.failures += 1
        self.last_failure_time = time.time()
        if self.failures >= self.fail_threshold:
            self.state = "open"

    def reset(self):
        self.failures = 0
        self.state = "closed"

def unreliable_service():
    """Simulate a call to an unreliable service."""
    import random
    if random.random() < 0.5:  # 50% chance of failure
        raise Exception("Service failure.")
    return "Service response"

breaker = CircuitBreaker()

@app.route('/')
def index():
    try:
        # Make a call through the circuit breaker
        response = breaker.call(unreliable_service)
        return jsonify({'response': response, 'status': 'success'})
    except Exception as e:
        return jsonify({'error': str(e), 'status': 'service unavailable'})

if __name__ == '__main__':
    app.run(port=5005)


- CircuitBreaker Class: La clase CircuitBreaker gestiona los estados del circuito. Mantiene un conteo de fallos y cambia el estado del circuito a "abierto" si el número de fallos consecutivos excede un umbral definido (fail_threshold). Si el circuito está en estado "abierto" y ha pasado suficiente tiempo (reset_timeout), el circuito se moverá al estado "semiabierto" en el siguiente intento de llamada.

- Función de servicio poco confiable: unreliable_service simula un servicio externo que podría fallar aleatoriamente, lo que es ideal para demostrar el comportamiento del circuit breaker.

- Flask App: La aplicación Flask expone una ruta que hace una llamada a través del circuit breaker. Si el circuit breaker está "abierto" y no se ha movido a "semiabierto" o "cerrado", se lanzará una excepción y se devolverá un error al cliente.


**Ejecución y prueba**

- Ejecutar el microservicio: Inicia la aplicación Flask ejecutando el script en tu entorno local. Esto iniciará el servidor en el puerto 5005.
- Probar el comportamiento: Usa un navegador o herramientas como curl o Postman para hacer solicitudes a http://localhost:5005/. Podrás observar cómo, después de varios fallos simulados, el circuit breaker se activa, rechazando las llamadas hasta que el tiempo de reinicio haya pasado, y luego intenta una llamada para ver si el servicio se ha recuperado.

python circuit_breaker_example.py

Abre un navegador web y visita http://localhost:5005/ o utiliza una herramienta como Postman o curl para hacer solicitudes al servidor:

curl http://localhost:5005/

Observa cómo el circuit breaker responde a fallos simulados y cómo maneja el estado abierto y semiabierto.

#### Tu participación


- ¿Qué representa el estado "abierto" en un circuit breaker y cómo afecta a las solicitudes subsecuentes en nuestro ejemplo?
- Explica el propósito del estado "semiabierto" en un circuit breaker. ¿Cómo decide el circuit breaker pasar del estado "semiabierto" a "cerrado"?
- Cambia el valor de fail_threshold a un número mayor o menor y observa cómo esto afecta la frecuencia con la que el circuito se abre. Documenta tus observaciones.
- Agrega una funcionalidad que registre cada cambio de estado en el circuit breaker a un archivo de log. Esto es útil para auditar y depurar el comportamiento del sistema.
- Modifica la función unreliable_service para que después de un número específico de fallos, garantice un éxito. Esto simulará un servicio que se recupera después de ciertas fallas. Observa cómo el circuit breaker maneja esta recuperación.
- Experimenta con diferentes valores para reset_timeout y evalúa cómo el tiempo de inactividad afecta la disponibilidad percibida del servicio. Considera escenarios de alta y baja carga en tus pruebas.

In [ ]:
import time
from flask import Flask, jsonify
import logging

app = Flask(__name__)


logging.basicConfig(filename='circuit_breaker.log', level=logging.INFO)

def record_failure(self):
    self.failures += 1
    self.last_failure_time = time.time()
    if self.failures >= self.fail_threshold:
        self.state = "open"
        logging.info(f'Circuit opened at {self.last_failure_time}')

class CircuitBreaker:
    def __init__(self, fail_threshold=3, reset_timeout=10):
        self.fail_threshold = fail_threshold
        self.reset_timeout = reset_timeout
        self.failures = 0
        self.last_failure_time = None
        self.state = "closed"
    
    def call(self, func, *args, **kwargs):
        if self.state == "open":
            if time.time() - self.last_failure_time > self.reset_timeout:
                self.state = "half-open"
            else:
                raise Exception("Circuit is open. Calls are not allowed.")
        
        try:
            result = func(*args, **kwargs)
            self.reset()
            return result
        except Exception as e:
            self.record_failure()
            raise e

    def record_failure(self):
        self.failures += 1
        self.last_failure_time = time.time()
        if self.failures >= self.fail_threshold:
            self.state = "open"

    def reset(self):
        self.failures = 0
        self.state = "closed"

failure_count = 0

def unreliable_service():
    global failure_count
    failure_count += 1
    if failure_count > 5:  # Garantiza un éxito después de 5 fallos
        failure_count = 0
        return "Service response"
    raise Exception("Service failure.")


breaker = CircuitBreaker(fail_threshold=3, reset_timeout=30)  # Incrementa o disminuye el timeout

@app.route('/')
def index():
    try:
        # Make a call through the circuit breaker
        response = breaker.call(unreliable_service)
        return jsonify({'response': response, 'status': 'success'})
    except Exception as e:
        return jsonify({'error': str(e), 'status': 'service unavailable'})

if __name__ == '__main__':
    app.run(port=5005)


### Ejemplo de implementación del Bulkhead

El patrón Bulkhead es utilizado en el diseño de sistemas para aislar fallos y limitar su propagación dentro de un sistema más grande. En el contexto de la programación paralela o distribuida, el patrón Bulkhead puede ser usado para limitar el número de recursos (como hilos o conexiones de red) que una parte del sistema puede consumir, previniendo así que un problema en una parte del sistema afecte al resto.

Para ilustrar el patrón Bulkhead en Python, podemos simular un escenario donde diferentes tareas o servicios compiten por recursos limitados. Utilizaremos el módulo concurrent.futures para manejar tareas asincrónicas con un número limitado de trabajadores y  limitar la cantidad de recursos utilizados por cada tarea.

In [ ]:
import concurrent.futures
import time

# Función que simula una tarea que puede fallar o tardar mucho
def task(id, duration):
    try:
        print(f"Iniciando tarea {id} que tardará {duration} segundos.")
        time.sleep(duration)  # Simula tiempo de procesamiento
        print(f"Tarea {id} completada.")
        return f"Resultado de la tarea {id}"
    except Exception as e:
        return f"Tarea {id} falló: {str(e)}"

# Función principal que ejecuta tareas utilizando un Bulkhead
def main():
    # Usar ThreadPoolExecutor como un Bulkhead
    # Limitamos el número de tareas simultáneas a 3
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        # Simulamos la recepción de tareas con diferentes duraciones
        tasks = {executor.submit(task, i, duration): i for i, duration in enumerate([2, 3, 5, 1, 4])}
        for future in concurrent.futures.as_completed(tasks):
            task_id = tasks[future]
            try:
                result = future.result()
                print(f"Tarea {task_id} completada con resultado: {result}")
            except Exception as e:
                print(f"Tarea {task_id} generó una excepción: {str(e)}")

if __name__ == "__main__":
    main()


Explicación del código:

- Función de tarea (task): Esta función representa una tarea individual que podría tardar un tiempo variable en completarse, simbolizado por la llamada a time.sleep(). La función imprime mensajes al comenzar y completar la tarea.
- Ejecución de Tareas (main): En la función principal, utilizamos ThreadPoolExecutor de concurrent.futures para simular el patrón Bulkhead. Se establece un máximo de tres trabajadores (hilos) para manejar las tareas, limitando así el número de tareas que se ejecutan en paralelo.
- Gestión de tareas: Las tareas se agregan al ejecutor y se monitorean hasta su completación. El executor.submit inicia la tarea, y as_completed maneja las tareas conforme van terminando. Cada tarea se identifica y se manejan excepciones potenciales que podrían surgir durante su ejecución.

Ejecución del código:

python bulkhead_example.py

* Asegúrate de usar python3 en lugar de python si tienes múltiples versiones de Python instaladas y python no se refiere a Python 3.x.
* Después de ejecutar el script, verás en la terminal la salida del programa, mostrando cuándo cada tarea comienza y termina, y cómo el sistema maneja múltiples tareas simultáneamente limitadas a un máximo de tres en ejecución paralela.
* La salida te dará una idea clara de cómo el patrón Bulkhead está limitando el número de tareas concurrentes para prevenir la sobrecarga del sistema y mejorar el manejo de recurso.

#### Tu participación

- Explica cómo el patrón Bulkhead contribuye a la resiliencia de un sistema de microservicios. ¿Cómo previene que fallos en una parte del sistema afecten al resto del sistema?
- Compara el patrón Bulkhead con el patrón Circuit Breaker. ¿En qué situaciones es preferible uno sobre el otro?
- Discute los posibles riesgos de no implementar el patrón Bulkhead en un sistema que maneja múltiples tareas o solicitudes concurrentes.
- ¿Cómo influiría la implementación del patrón Bulkhead en el diseño de una nueva aplicación de software? Considera aspectos como la escalabilidad, mantenibilidad y la gestión de fallos.
- Modifica el número de trabajadores (max_workers) en el ThreadPoolExecutor y observa cómo afecta el rendimiento y la capacidad de respuesta del sistema. Documenta los resultados con diferentes configuraciones.
- Mejora el manejo de excepciones en el código para asegurarte de que ninguna tarea pueda causar un fallo en el sistema completo. Implementa registros detallados de los errores para facilitar la depuración.
- Escribe un script que genere un número muy alto de tareas simultáneas y utiliza el patrón Bulkhead para manejar adecuadamente esta carga. Observa y documenta cómo el sistema maneja la sobrecarga.
- Integra funcionalidades de monitorización para rastrear el tiempo de ejecución de cada tarea y el uso de recursos del sistema. Utiliza esta información para ajustar dinámicamente el número de hilos en el ThreadPoolExecutor.
- Combina el patrón Bulkhead con el patrón Circuit Breaker para manejar mejor las tareas que fallan repetidamente. Implementa lógica para que, después de cierto número de fallos consecutivos, el sistema deje de intentar ejecutar una tarea fallida durante un período de tiempo.

In [1]:
import logging
import time
import concurrent.futures

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def task(id, duration):
    logging.info(f"Iniciando tarea {id} que tardará {duration} segundos.")
    time.sleep(duration)
    if id % 2 == 0:
        raise Exception(f"Tarea {id} falló.")
    logging.info(f"Tarea {id} completada.")
    return f"Resultado de la tarea {id}"

def main():
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futures = []
        for i in range(6):
            future = executor.submit(task, i, i+1)
            futures.append(future)
        
        for future in concurrent.futures.as_completed(futures):
            try:
                result = future.result()
                logging.info(result)
            except Exception as e:
                logging.error(str(e))

if __name__ == "__main__":
    main()


2024-05-09 22:45:26,018 - INFO - Iniciando tarea 0 que tardará 1 segundos.
2024-05-09 22:45:26,022 - INFO - Iniciando tarea 1 que tardará 2 segundos.
2024-05-09 22:45:26,023 - INFO - Iniciando tarea 2 que tardará 3 segundos.
2024-05-09 22:45:26,027 - INFO - Iniciando tarea 3 que tardará 4 segundos.
2024-05-09 22:45:26,028 - INFO - Iniciando tarea 4 que tardará 5 segundos.
2024-05-09 22:45:27,021 - INFO - Iniciando tarea 5 que tardará 6 segundos.
2024-05-09 22:45:27,022 - ERROR - Tarea 0 falló.
2024-05-09 22:45:28,025 - INFO - Tarea 1 completada.
2024-05-09 22:45:28,025 - INFO - Resultado de la tarea 1
2024-05-09 22:45:29,026 - ERROR - Tarea 2 falló.
2024-05-09 22:45:30,041 - INFO - Tarea 3 completada.
2024-05-09 22:45:30,044 - INFO - Resultado de la tarea 3
2024-05-09 22:45:31,053 - ERROR - Tarea 4 falló.
2024-05-09 22:45:33,026 - INFO - Tarea 5 completada.
2024-05-09 22:45:33,027 - INFO - Resultado de la tarea 5


### Ejemplo de implementación del Sidecar 
En este ejemplo, tendrás un servicio principal que realiza alguna funcionalidad básica y un sidecar que se encargará de hacer el logging de las actividades del servicio principal.

El servicio principal será una simple aplicación Flask que expone un endpoint API.


In [2]:
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return jsonify({"message": "Hello from the main service!"})

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


2024-05-09 22:45:39,630 - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2024-05-09 22:45:39,631 - INFO - Press CTRL+C to quit
2024-05-09 22:45:42,255 - INFO - 127.0.0.1 - - [09/May/2024 22:45:42] "GET / HTTP/1.1" 200 -


El sidecar será responsable de revisar y registrar las solicitudes realizadas al servicio principal. Utilizaremos aiohttp para crear un pequeño servidor web que actúe como un proxy, registrando cada solicitud que pase hacia el servicio principal.

In [5]:
from flask import Flask, jsonify
import aiohttp
from aiohttp import web
import asyncio

async def handler(request):
    async with aiohttp.ClientSession() as session:
        # Proxear la solicitud al servicio principal
        async with session.get('http://localhost:5000') as response:
            data = await response.json()
            print(f"Logging request: {data}")  # Simula el registro de la solicitud
            return web.json_response(data)

app = web.Application()
app.router.add_route('GET', '/', handler)

if __name__ == '__main__':
    web.run_app(app, port=5001)


RuntimeError: Cannot run the event loop while another loop is running

Explicación del código:

- Servicio principal: Se ejecuta en el puerto 5000 y simplemente devuelve un mensaje de bienvenida. Este servicio representa cualquier microservicio en una arquitectura más grande que realiza operaciones específicas del negocio.

- Sidecar: Se ejecuta en el puerto 5001 y actúa como un proxy hacia el servicio principal. Captura cada solicitud y realiza el registro (logging) antes de pasar la solicitud al servicio principal. Este ejemplo ilustra cómo un sidecar puede ser usado para offload tareas como logging y monitorización, liberando al servicio principal para que se enfoque en sus funcionalidades principales.

Ejecución y prueba del sistema

- Ejecutar el servicio principal: Guarda el código en un archivo, por ejemplo main_service.py, y ejecútalo usando Python.
- Ejecutar el Sidecar: Guarda el código del sidecar en otro archivo, por ejemplo sidecar.py, y ejecútalo en un terminal diferente.

Para probar que todo está funcionando, puedes hacer una solicitud HTTP al sidecar:
 curl http://localhost:5001/

Deberías ver que el sidecar registra la solicitud y luego pasa la respuesta del servicio principal.

**Observación:** Este es un ejemplo muy simplificado del patrón Sidecar. En un entorno real, especialmente en entornos basados en contenedores como Kubernetes, los sidecars pueden ser más complejos y manejar tareas más críticas, aprovechando la proximidad al servicio principal para mejorar la eficiencia y la seguridad.

#### Tu participación

- ¿Qué es el patrón Sidecar y cómo se diferencia de otros patrones arquitectónicos en microservicios?
- ¿Cuáles son las ventajas de usar un Sidecar en comparación con incorporar funcionalidades directamente en el servicio principal?
- Identifica y describe tres escenarios de uso real donde el patrón Sidecar podría ser especialmente beneficioso en una arquitectura de microservicios.
- Discute cómo el patrón Sidecar puede mejorar la resiliencia y la escalabilidad de una aplicación distribuida.
- ¿Cómo se implementa típicamente el patrón Sidecar en entornos basados en contenedores como Kubernetes? Describe el proceso y los beneficios.
- Basándote en el ejemplo proporcionado, expande el sidecar para que no solo registre las solicitudes, sino también las respuestas del servicio principal. Asegúrate de capturar detalles como el tiempo de respuesta y el código de estado.
- Desarrolla un Sidecar que periódicamente verifique la salud del servicio principal y registre el estado. Implementa notificaciones (por ejemplo, a través de un simple print o guardando en un archivo) si el servicio principal no responde adecuadamente.
- Modifica el Sidecar para que simule la introducción de fallas en el servicio principal bajo ciertas condiciones (por ejemplo, cada diez solicitudes) y prueba cómo el sistema maneja y se recupera de tales fallas.
- Crea un Sidecar que funcione como un proxy avanzado, implementando características como la autenticación y el balanceo de carga entre múltiples instancias del servicio principal.
- Implementa un Sidecar que cache las respuestas del servicio principal para mejorar la eficiencia y reducir la carga. El Sidecar debería verificar si una solicitud tiene una respuesta válida en caché y servirla inmediatamente sin redirigir la solicitud al servicio principal.

In [ ]:
#salud
import requests
import time

def check_health(service_url):
    try:
        response = requests.get(service_url)
        if response.status_code == 200:
            print("Servicio principal en buen estado.")
        else:
            print("Servicio principal con problemas.")
    except requests.exceptions.RequestException:
        print("Servicio principal no accesible.")

def main():
    service_url = 'http://localhost:5000/health'
    while True:
        check_health(service_url)
        time.sleep(10)  # Verifica cada 10 segundos

if __name__ == "__main__":
    main()

#fallas
import requests
import random

def possibly_fail_request(service_url):
    if random.randint(1, 10) == 1:  # 10% de probabilidad de falla
        print("Introduciendo una falla en el sistema.")
        requests.get(service_url + "/fail")  # Endpoint que causa una falla
    else:
        response = requests.get(service_url)
        print("Respuesta normal del servicio principal.")

def main():
    service_url = 'http://localhost:5000'
    while True:
        possibly_fail_request(service_url)

if __name__ == "__main__":
    main()

#3. Sidecar como Proxy Avanzado
from flask import Flask, request, jsonify
import requests
import itertools

app = Flask(__name__)
services = ['http://localhost:5001', 'http://localhost:5002']
service_cycle = itertools.cycle(services)

@app.route('/<path:path>', methods=['GET', 'POST'])
def proxy(path):
    service_url = next(service_cycle)
    if request.method == 'GET':
        resp = requests.get(f'{service_url}/{path}')
    elif request.method == 'POST':
        resp = requests.post(f'{service_url}/{path}', json=request.get_json())
    return jsonify(resp.json())

if __name__ == "__main__":
    app.run(port=5003)

#4 catching
from flask import Flask, request, jsonify
import requests

app = Flask(__name__)
cache = {}

@app.route('/<path:path>', methods=['GET'])
def cached_proxy(path):
    if path in cache:
        print("Sirviendo desde caché.")
        return jsonify(cache[path])
    else:
        response = requests.get(f'http://localhost:5000/{path}')
        cache[path] = response.json()
        print("Caché actualizado y sirviendo respuesta fresca.")
        return jsonify(cache[path])

if __name__ == "__main__":
    app.run(port=5004)


### Ejemplo de implementación de Edge Server

Para ilustrar el patrón de servidor perimetral (Edge Server) utilizando Python, podemos utilizar Flask para crear un simple servidor perimetral que actúa como un proxy inverso y también Flask para los microservicios internos. Vamos a simular un escenario donde el servidor perimetral controla el acceso a tres microservicios, protegiéndolos de solicitudes externas no autorizadas y realizando un balanceo de carga simple.

In [ ]:
from flask import Flask, request, jsonify, abort
import requests

app = Flask(__name__)

# Configuración de los microservicios internos
services = {
    'microservice_a': 'http://localhost:5001',
    'microservice_b': 'http://localhost:5002',
    'microservice_c': 'http://localhost:5003'
}

# Función de autenticación de ejemplo
def authenticate_request(api_key):
    # Simular la autenticación con una clave API
    return api_key == "secret"

@app.route('/<service>', methods=['GET', 'POST'])
def proxy(service):
    if service not in services:
        return jsonify({"error": "Service not found"}), 404

    if not authenticate_request(request.headers.get('X-API-Key')):
        return jsonify({"error": "Unauthorized"}), 401

    service_url = services[service]
    response = requests.request(
        method=request.method,
        url=f"{service_url}{request.full_path}",
        headers={key: value for key, value in request.headers if key != 'Host'},
        data=request.get_data(),
        allow_redirects=False)

    return (response.content, response.status_code, response.headers.items())

if __name__ == '__main__':
    app.run(port=5000)


Aquí hay un ejemplo de cómo podría lucir un microservicio:

In [ ]:
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return jsonify({"message": "Hello from Microservice A"})

if __name__ == '__main__':
    app.run(port=5001)  # Cambia el puerto para B y C


Repite el mismo código para Microservice B y C, cambiando el mensaje y el puerto (5002 para B, 5003 para C).

Explicación del código

- Servidor Perimetral: El servidor perimetral actúa como un proxy inverso, manejando todas las solicitudes entrantes. Solo redirige solicitudes a servicios específicos configurados y autentica las solicitudes utilizando una clave API simple.
- Autenticación: Se realiza mediante la función authenticate_request, que comprueba si la clave API proporcionada en la cabecera X-API-Key es válida.
- Proxying Requests: Las solicitudes se reenvían al microservicio correspondiente, incluyendo la manipulación de los métodos HTTP, parámetros y cabeceras.
- Microservicios internos: Cada microservicio simplemente devuelve un mensaje. Estos servicios están configurados para ejecutarse en diferentes puertos y no son accesibles directamente desde el exterior sin pasar por el servidor perimetral.

**Implementación del código**

- Guarda el código del servidor perimetral en un archivo, por ejemplo, edge_server.py.
- Guarda el código de cada microservicio en archivos separados, por ejemplo, microservice_a.py, microservice_b.py, y microservice_c.py.
- Ejecuta el servidor perimetral:
  python edge_server.py
    Esto iniciará el servidor perimetral en localhost en el puerto 5000.
- Abre terminales separadas para cada microservicio.
- Navega al directorio donde están guardados los archivos de los microservicios.
- Ejecuta cada microservicio en un puerto diferente:

  ```
  python microservice_a.py  # se ejecutará en el puerto 5001
  python microservice_b.py  # se ejecutará en el puerto 5002
  python microservice_c.py  # se ejecutará en el puerto 5003
  ```

**Probar el sistema** 

* Puedes usar un navegador web o una herramienta como cURL para hacer solicitudes al servidor perimetral. Asegúrate de incluir la clave API correcta en las cabeceras de la solicitud para autenticar:

 ```
   curl -H "X-API-Key: secret" http://localhost:5000/microservice_a
   curl -H "X-API-Key: secret" http://localhost:5000/microservice_b
   curl -H "X-API-Key: secret" http://localhost:5000/microservice_c
 ```
* Intenta enviar solicitudes sin la clave API o con una clave incorrecta para ver cómo el servidor perimetral maneja la autenticación:

  ```
    curl http://localhost:5000/microservice_a  # Debería fallar con 401 Unauthorized
  ```

#### Tu participación

- ¿Qué funciones realiza un servidor perimetral (Edge Server) en una arquitectura de microservicios? ¿Por qué es crucial para la seguridad y la gestión del tráfico?
- Compara el servidor perimetral con el API Gateway. ¿Cuáles son las similitudes y diferencias clave entre estos dos patrones?
- ¿Cómo puede un servidor perimetral ayudar a prevenir ataques comunes como DDoS, SQL Injection, y Cross-Site Scripting (XSS)?
- ¿Cómo contribuye un servidor perimetral al escalado de aplicaciones en un entorno de microservicios? Considera escenarios con alto . - --- ¿Cómo puede diseñarse un servidor perimetral para manejar fallas en los microservicios a los que redirige? Considera técnicas como el - 
- Modifica el servidor perimetral para utilizar JSON Web Tokens (JWT) en lugar de claves API simples para la autenticación. Implementa un endpoint que emita tokens JWT a los clientes después de una autenticación exitosa basada en nombre de usuario y contraseña.
- Agrega capacidades de logging avanzadas al servidor perimetral. Asegúrate de registrar todas las solicitudes entrantes, incluyendo detalles como la dirección IP del solicitante, la hora de la solicitud, y la respuesta del microservicio. Configura el logging para que los datos se escriban en un archivo en un formato que facilite el análisis posterior.
- Implementa una funcionalidad en el servidor perimetral que permita modificar dinámicamente las rutas o los microservicios a los que se redirigen las solicitudes sin necesidad de reiniciar el servidor. Esto podría involucrar la lectura de un archivo de configuración o consultar una base de datos de rutas en tiempo real.
- [Opcional] Escribe un middleware para el servidor perimetral que analice la carga de tráfico y tome decisiones inteligentes sobre cuándo rechazar nuevas conexiones o redirigirlas a otros servidores para balancear la carga.
- [Opcional] Desarrolla un script que simule un ataque DDoS al servidor perimetral y luego mejora el servidor perimetral para manejar eficazmente tales ataques, por ejemplo, implementando límites de tasa o desafíos CAPTCHA.

In [ ]:
# Implementación de JWT y Endpoint de Autenticación
from flask import Flask, jsonify, request
import jwt
import datetime

app = Flask(__name__)
SECRET_KEY = "your_secret_key"

@app.route('/token', methods=['POST'])
def create_token():
    username = request.json.get('username')
    password = request.json.get('password')
    if username == "admin" and password == "password":  # Esta es una simplificación
        payload = {
            'exp': datetime.datetime.utcnow() + datetime.timedelta(hours=24),
            'iat': datetime.datetime.utcnow(),
            'sub': username
        }
        token = jwt.encode(payload, SECRET_KEY, algorithm='HS256')
        return jsonify({'token': token})
    else:
        return jsonify({'message': 'Invalid credentials'}), 401

if __name__ == '__main__':
    app.run(debug=True)


### Ejemplo de implementación de microservicios reactivos

Primero, necesitas instalar las bibliotecas necesarias. Si no tienes instalado aiohttp, puedes hacerlo con pip:

pip install aiohttp

Presentamos un ejemplo de cómo podrías implementar un microservicio utilizando aiohttp para manejar las solicitudes de forma asincrónica:

In [ ]:
from aiohttp import web, ClientSession
import asyncio

async def fetch_data(url):
    """Función asincrónica para obtener datos de otro servicio."""
    async with ClientSession() as session:
        async with session.get(url) as response:
            return await response.text()

async def handle_request(request):
    """Manejador que simula la llamada a otro microservicio."""
    data_url = "http://example.com/data"  # URL del servicio de datos
    data = await fetch_data(data_url)
    return web.Response(text=f"Data from other service: {data}")

async def service_health(request):
    """Endpoint para chequear la salud del servicio."""
    return web.Response(text="Service is up and running!")

app = web.Application()
app.add_routes([web.get('/', handle_request),
                web.get('/health', service_health)])

if __name__ == '__main__':
    web.run_app(app, port=8080)


Descripción del código

- fetch_data: Esta función asincrónica realiza solicitudes HTTP a otro servicio. Utiliza aiohttp.ClientSession para manejar las conexiones de manera eficiente y asincrónica.
- handle_request: Es un manejador para las solicitudes entrantes que, a su vez, hace una llamada a otro microservicio. La llamada es no bloqueante y espera asincrónicamente por la respuesta.
- service_health: Un simple endpoint para verificar la salud del servicio, útil para implementaciones de microservicios que necesitan ser monitoreados y mantenidos.
- Se configura una aplicación de aiohttp y se añaden rutas para manejar las solicitudes.

Ejecución del código

- Abre una terminal en la carpeta donde guardaste el archivo reactive_microservice.py.
- Ejecuta el archivo con Python mediante el siguiente comando:

  python reactive_microservice.py

Este comando iniciará el servidor web en el puerto 8080.

Una vez que el servidor esté en ejecución, puedes probar el microservicio utilizando un navegador o una herramienta como curl. Aquí tienes cómo puedes hacerlo:

- Probar el endpoint principal:Abre tu navegador y visita http://localhost:8080/ o usa curl en la terminal:
  curl http://localhost:8080/

Esto debería ejecutar la función handle_request que a su vez hará una llamada al URL especificado en data_url (asegúrate de que data_url apunte a un servicio válido o simula uno para pruebas).

- Probar el endpoint de salud:Puedes verificar si el servicio está funcionando correctamente visitando http://localhost:8080/health o usando curl:

    curl http://localhost:8080/health

Esto debería devolver el mensaje "Service is up and running!"

Consideraciones

- Asegúrate de que el puerto 8080 esté libre o cambia el puerto en el código si es necesario.
- El URL data_url en la función fetch_data debe apuntar a un destino válido que responda a solicitudes HTTP GET, o debes simular uno para pruebas.


#### Tu participación

- ¿Cómo contribuye el modelo asincrónico a la resiliencia y escalabilidad de los microservicios comparado con un modelo sincrónico tradicional?
- Explica cómo los microservicios pueden ser diseñados para ser autorreparables y qué estrategias se podrían implementar para lograrlo.
- Modifica el microservicio para que haga llamadas paralelas a múltiples servicios en handle_request, utilizando asyncio.gather para combinar los resultados y responder una vez que todas las respuestas estén disponibles.
- Implementa un mecanismo de reintento en fetch_data para manejar fallos temporales, donde el servicio intenta nuevamente la solicitud después de un breve retraso.

In [ ]:
## Llamadas Paralelas con asyncio.gather
async def handle_request(request):
    """Manejador que realiza llamadas paralelas a múltiples servicios."""
    service_urls = ["http://example.com/data1", "http://example.com/data2"]
    responses = await asyncio.gather(*(fetch_data(url) for url in service_urls))
    combined_data = " ".join(responses)
    return web.Response(text=f"Combined data from services: {combined_data}")

##Mecanismo de Reintento en fetch_data
async def fetch_data(url, attempts=3, delay=2):
    """Función asincrónica que reintenta obtener datos de otro servicio en caso de fallo."""
    async with ClientSession() as session:
        try:
            async with session.get(url, timeout=10) as response:
                response.raise_for_status()
                return await response.text()
        except Exception as e:
            if attempts > 1:
                await asyncio.sleep(delay)  # Esperar antes de reintentar
                return await fetch_data(url, attempts - 1, delay * 2)  # Aumentar el retraso
            else:
                return f"Failed after retries: {str(e)}"









### Ejemplo de implementación de distributed tracing 

Vamos a crear tres microservicios simples que se llamarán entre sí. Cada uno registrará sus acciones junto con un ID de correlación.

In [ ]:
from flask import Flask, request, jsonify
import requests
import uuid
import logging

# Configuración de logging
logging.basicConfig(level=logging.INFO)

app = Flask(__name__)

def get_correlation_id():
    # Obtener el correlation ID de la cabecera de la solicitud, o generar uno nuevo si no existe
    return request.headers.get('X-Correlation-ID', uuid.uuid4().hex)

@app.route('/serviceA')
def service_a():
    correlation_id = get_correlation_id()
    logging.info(f"Service A called with correlation ID: {correlation_id}")

    # Llamada a Service B
    headers = {'X-Correlation-ID': correlation_id}
    response_b = requests.get("http://localhost:5001/serviceB", headers=headers)
    
    # Llamada a Service C
    response_c = requests.get("http://localhost:5002/serviceC", headers=headers)
    
    return jsonify({
        'serviceB_response': response_b.text, 
        'serviceC_response': response_c.text
    })

@app.route('/serviceB')
def service_b():
    correlation_id = get_correlation_id()
    logging.info(f"Service B called with correlation ID: {correlation_id}")
    return f"Service B processed with correlation ID: {correlation_id}"

@app.route('/serviceC')
def service_c():
    correlation_id = get_correlation_id()
    logging.info(f"Service C called with correlation ID: {correlation_id}")
    return f"Service C processed with correlation ID: {correlation_id}"

if __name__ == '__main__':
    # Asumiendo diferentes puertos para simulación de diferentes servicios
    import sys
    port = 5000 if len(sys.argv) == 1 else int(sys.argv[1])
    app.run(port=port)


Explicación del código

- Utilizamos el módulo logging de Python para registrar los eventos. Cada evento registrará el ID de correlación para rastrear la solicitud a través de los servicios.
- En cada servicio, intentamos recuperar el X-Correlation-ID de las cabeceras HTTP. Si no está presente, generamos uno nuevo utilizando uuid.uuid4().hex. Esto asegura que cada cadena de solicitudes tenga un ID único que las trace.
- El ServiceA hace llamadas a ServiceB y ServiceC, pasando el X-Correlation-ID en las cabeceras HTTP para mantener la trazabilidad.


Ejecución y Pruebas

- Guarda el código en un archivo, por ejemplo, tracing_example.py.
- Ejecuta tres instancias del servidor en diferentes terminales para simular los diferentes servicios:

    python tracing_example.py 5000
    python tracing_example.py 5001
    python tracing_example.py 5002

Usa un navegador o una herramienta como curl para llamar al ServiceA y observar las respuestas:

    curl http://localhost:5000/serviceA



Verás en los logs que cada llamada a los servicios incluye el mismo ID de correlación, lo que te permite rastrear cómo se procesa una solicitud a través de diferentes servicios. 

#### Tu participación

- ¿Qué es un ID de correlación y cómo facilita el seguimiento distribuido en arquitecturas de microservicios?
- Explica cómo la propagación del ID de correlación entre los servicios mejora la capacidad de depuración y monitorización en sistemas distribuidos.
- ¿Cuáles son las ventajas de utilizar un sistema centralizado de logging en un entorno de microservicios?
- Discute los desafíos de implementar el seguimiento distribuido en un sistema de microservicios que ya está en producción. ¿Qué estrategias podrían mitigar estos desafíos?
- ¿Cómo puede impactar el seguimiento distribuido en el rendimiento de un sistema de microservicios y cómo se podría minimizar este impacto?
- Modifica el código para incluir más información en los logs, como parámetros específicos de la solicitud o más detalles del estado interno en cada punto de logging. Ejemplo: Registrar los parámetros de entrada en ServiceB y ServiceC.
- Agrega código que simule un fallo aleatorio en ServiceC.
- Implementa una solución que reintente la solicitud a ServiceC si falla, utilizando el mismo ID de correlación y registra cada intento.
- Escribe un script que extraiga logs de un archivo y filtre por ID de correlación. Este script debería ser capaz de mostrar la cadena completa de eventos para una solicitud específica. Ejemplo: Un script Python que lee un archivo de log y muestra todas las entradas relacionadas con un ID de correlación dado.
- Agrega manejo de timeouts a las solicitudes HTTP en ServiceA que se hacen a ServiceB y ServiceC.
- Modifica el código para registrar la latencia de las solicitudes entre servicios. Registra el tiempo que toma cada servicio para responder y analiza los datos para identificar posibles cuellos de botella.

In [ ]:
## Mejora del Sistema de Logging

@app.route('/serviceB')
def service_b():
    correlation_id = get_correlation_id()
    param = request.args.get('param', 'default')
    logging.info(f"Service B called with correlation ID: {correlation_id}, Parameter: {param}")
    return f"Service B processed with correlation ID: {correlation_id}, Parameter: {param}"

@app.route('/serviceC')
def service_c():
    correlation_id = get_correlation_id()
    status = "Starting process"
    logging.info(f"Service C called with correlation ID: {correlation_id}, Status: {status}")
    try:
        # Simular fallo aleatorio
        if random.random() < 0.5:
            raise ValueError("Simulated Failure")
        status = "Completed successfully"
    except Exception as e:
        status = f"Failed with error: {str(e)}"
        logging.error(f"Service C error with correlation ID: {correlation_id}: {str(e)}")
    finally:
        logging.info(f"Service C final status with correlation ID: {correlation_id}: {status}")
    return f"Service C processed with correlation ID: {correlation_id}, Status: {status}"

## Implementación de Retries

def retry_request(url, headers, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response.text
        except requests.RequestException as e:
            logging.error(f"Attempt {attempt + 1}: Failed to call {url} with error: {str(e)}")
            time.sleep(1)  # Espera antes de reintentar
    return "Failed after retries"

@app.route('/serviceA')
def service_a():
    correlation_id = get_correlation_id()
    headers = {'X-Correlation-ID': correlation_id}
    response_b = requests.get("http://localhost:5001/serviceB", headers=headers)
    response_c = retry_request("http://localhost:5002/serviceC", headers)
    return jsonify({
        'serviceB_response': response_b.text, 
        'serviceC_response': response_c
    })

#Monitorización de Latencia

@app.route('/serviceA')
def service_a():
    start_time = time.time()
    correlation_id = get_correlation_id()
    headers = {'X-Correlation-ID': correlation_id}
    response_b = requests.get("http://localhost:5001/serviceB", headers=headers)
    response_c = retry_request("http://localhost:5002/serviceC", headers)
    total_time = time.time() - start_time
    logging.info(f"Total processing time for correlation ID {correlation_id}: {total_time} seconds")
    return jsonify({
        'serviceB_response': response_b.text, 
        'serviceC_response': response_c,
        'latency': total_time
    })

#Script de Extracción de Logs

def filter_logs_by_correlation_id(file_path, correlation_id):
    with open(file_path, 'r') as file:
        for line in file:
            if correlation_id in line:
                print(line)

# Uso del script
filter_logs_by_correlation_id('path_to_log_file.log', 'desired_correlation_id')

